<a href="https://colab.research.google.com/github/Magesh-Pragmath/Text-Classification-Demo/blob/master/Training_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [6]:
from transformers import pipeline
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict

In [7]:
classifier = pipeline("text-classification", model='distilbert-base-uncased-finetuned-sst-2-english')

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
nlp=classifier("I've been waiting for a HuggingFace course my whole life.")
print(nlp)

[{'label': 'POSITIVE', 'score': 0.9598046541213989}]


In [9]:
result=classifier("I relly enjoyed studying NLP")
print(result)

[{'label': 'POSITIVE', 'score': 0.9974174499511719}]


In [10]:
from datasets import load_dataset

ddict = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train_df.csv:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

val_df.csv:   0%|          | 0.00/601k [00:00<?, ?B/s]

test_df.csv:   0%|          | 0.00/586k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

In [11]:
df = ddict['train'].to_pandas()

In [12]:
ddict

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 31232
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5206
    })
})

In [13]:
df.head()

,id,text,label,sentiment
0,9536,"Cooking microwave pizzas, yummy",2,positive
1,6135,Any plans of allowing sub tasks to show up in ...,1,neutral
2,17697,"I love the humor, I just reworded it. Like sa...",2,positive
3,14182,naw idk what ur talkin about,1,neutral
4,17840,That sucks to hear. I hate days like that,0,negative


In [14]:
df.drop(columns=["label"], axis=1, inplace=True)

In [15]:
df.head()

,id,text,sentiment
0,9536,"Cooking microwave pizzas, yummy",positive
1,6135,Any plans of allowing sub tasks to show up in ...,neutral
2,17697,"I love the humor, I just reworded it. Like sa...",positive
3,14182,naw idk what ur talkin about,neutral
4,17840,That sucks to hear. I hate days like that,negative


In [16]:
label=LabelEncoder()
df['sentiment'] = label.fit_transform(df['sentiment'])

In [17]:
df.head()

,id,text,sentiment
0,9536,"Cooking microwave pizzas, yummy",2
1,6135,Any plans of allowing sub tasks to show up in ...,1
2,17697,"I love the humor, I just reworded it. Like sa...",2
3,14182,naw idk what ur talkin about,1
4,17840,That sucks to hear. I hate days like that,0


In [18]:
train, validate, test = np.split(df.sample(frac=1, random_state=42),[int(.6*len(df)),int(.8*len(df))])

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [19]:
train = train.reset_index()[['text','sentiment']]
validate = validate.reset_index()[['text','sentiment']]
test = test.reset_index()[['text','sentiment']]

In [20]:
train.head()

,text,sentiment
0,@mashable I never did thank you for including ...,2
1,Happy Mothers Day!!,2
2,Welcome home! Glad you made it home safe!,2
3,@__Jasper_Hale__ *Laughs* that`s because I lik...,2
4,Getting ready for work... think I`m catching a...,1


In [21]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
teds = Dataset.from_pandas(test)

In [22]:
dataset = DatasetDict()
dataset["train"] = tds
dataset["validate"] = vds
dataset["test"] = teds

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 18739
    })
    validate: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 6246
    })
    test: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 6247
    })
})

In [24]:
from transformers import AutoTokenizer

model_cpkt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_cpkt)

In [25]:
def tokenize(batch):
  return tokenizer(batch["text"],padding=True, truncation=True)

In [26]:
dataset_encode = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/18739 [00:00<?, ? examples/s]

Map:   0%|          | 0/6246 [00:00<?, ? examples/s]

Map:   0%|          | 0/6247 [00:00<?, ? examples/s]